In [2]:
%config Completer.use_jedi = False
%load_ext autoreload
%autoreload 2

In [27]:
import os, sys
import seaborn as sns
import numpy as np
from collections import defaultdict

In [4]:
import matplotlib.pyplot as plt

sys.path.append('../experimental_sripts')

sys.path.append('/home/Container/tgnn/scripts/Models/BGNN_extended.py')

import pandas as pd
import json

from run import aggregate_results, get_name

Using backend: pytorch


In [5]:
with open('/home/Container/tgnn/scripts/experimental_sripts/results4/house/MGBDT/seed_results.json', 'r') as fp:
    MGBDT_seed = json.load(fp)

In [48]:
def filter_dict(ddict, filt):
    return {key:ddict[key] for key in filter(lambda x: filt in x, ddict)}

In [49]:
house_results = aggregate_results('../experimental_sripts/results4/house/MGBDT/seed_results.json', 'regression', filt='pretrain_option')

In [50]:
house_results = filter_dict(house_results, '[6, 4]')

In [51]:
vk_results = aggregate_results('../experimental_sripts/results4/vk/MGBDT/seed_results.json',\
                  'regression', filt='pretrain_option')

In [52]:
vk_results = filter_dict(vk_results, '[14, 4]')

In [55]:
county_results = aggregate_results('../experimental_sripts/results4/county/MGBDT/seed_results.json',\
                  'regression', filt='pretrain_option')

In [56]:
county_results = filter_dict(county_results, '[7, 4]')

In [58]:
comparison_data = {'county':county_results,
                  'vk':vk_results,
                  'house':house_results}

In [64]:
strfy = lambda x: f'{x:.3f}'

In [70]:
comparison_dataset = defaultdict(list)
for dataset in comparison_data:
    for key, (mean, std) in comparison_data[dataset].items():
        options = key.split('_')
        comparison_dataset['pre_learn_option'] += [options[-2]\
                                        if eval(options[-1]) else 'None']
        comparison_dataset['dataset'] += [dataset]
        comparison_dataset['rmse'] += [strfy(mean) + ' ± ' + strfy(std)]

In [76]:
comparison_dataset = pd.DataFrame(comparison_dataset)

In [79]:
def highlight_min(s):
#     import pdb; pdb.set_trace()
    s = s.apply(lambda x : float(x.split(' ')[0]))
    return ['color: green' if i == 1 else 'color: balck' for i in s.rank(method='dense')]

In [80]:
comparison_dataset.pivot('pre_learn_option', 'dataset', 'rmse').style.apply(highlight_min)

dataset,county,house,vk
pre_learn_option,,,
None,1.285 ± 0.107,0.548 ± 0.019,6.970 ± 0.194
linear,1.267 ± 0.087,0.521 ± 0.008,7.004 ± 0.183
stack,1.249 ± 0.081,0.518 ± 0.008,6.915 ± 0.227
